# Aufgabe 30 Methode der kleinsten Quadrate

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## a) Designmatrix A  

hier $\text{Asymmetrie} = f(\Psi) = a_1 f_1(\Psi) + a_2 f_2(\Psi)$  
mit $f_1(\Psi) = \cos(\Psi)$, $f_2(\Psi) = \sin(\Psi)$  
Designmatrix:  $A = 
\begin{pmatrix}
    f_1(\Psi_1) & f_2(\Psi_1) & ... \\
    f_1(\Psi_2) & f_2(\Psi_2) & ... \\
    ... & ... & ...
\end{pmatrix}$

In [2]:
psi = np.arange(0,331,30) # in Grad
psi = np.radians(psi) # in rad
asym = [-0.032,0.010,0.057,
        0.068,0.076,0.080,
        0.031,0.005,-0.041,
        -0.090,-0.088,-0.074]
asym = np.array(asym)
asym_err = 0.011

print(f'{psi.shape = }')
print(f'{asym.shape = }')

psi.shape = (12,)
asym.shape = (12,)


In [3]:
# design matrix:
A = np.column_stack([np.cos(psi),np.sin(psi)])

print('Designmatrix A:')
print(f'{A.shape = }')
print(f'{A = }')

Designmatrix A:
A.shape = (12, 2)
A = array([[ 1.00000000e+00,  0.00000000e+00],
       [ 8.66025404e-01,  5.00000000e-01],
       [ 5.00000000e-01,  8.66025404e-01],
       [ 6.12323400e-17,  1.00000000e+00],
       [-5.00000000e-01,  8.66025404e-01],
       [-8.66025404e-01,  5.00000000e-01],
       [-1.00000000e+00,  1.22464680e-16],
       [-8.66025404e-01, -5.00000000e-01],
       [-5.00000000e-01, -8.66025404e-01],
       [-1.83697020e-16, -1.00000000e+00],
       [ 5.00000000e-01, -8.66025404e-01],
       [ 8.66025404e-01, -5.00000000e-01]])


## b) Lösungsvektor $\hat{\vec{a}}$

$\hat{\vec{a}} = (A^T A)^{-1} A^T \vec{y}$  
mit $\vec{y} = \text{Asymmetrien}$

In [4]:
y = asym
a = np.linalg.inv(A.T @ A) @ A.T @ y

print(f'{a.shape = }')
print(f'{a = }')

a.shape = (2,)
a = array([-0.0375063 ,  0.07739978])


## c) Kovarianzmatrix  $V[\hat{\vec{a}}]$
$V[\vec{y}] = \sigma^2 1$  
$V[\hat{\vec{a}}] = \sigma^2 (A^T A)^{-1}$

In [5]:
sigma = asym_err

In [6]:
# Kovarianzmatrix:
V_a = sigma**2 * np.linalg.inv(A.T @ A) 

print('Kovarianzmatrix von a:')
print(f'{V_a.shape = }')
print(f'{V_a = }')

Kovarianzmatrix von a:
V_a.shape = (2, 2)
V_a = array([[ 2.01666667e-05, -1.31944981e-21],
       [-1.31944981e-21,  2.01666667e-05]])


In [7]:
# Fehler von a:
a_err = np.sqrt(np.diag(V_a)) 
# Korrelationskoeffizient:
rho = V_a[1,0] / a_err[0]**2 

print(f'Fehler von a = {a_err}')
print(f'Korrelationskoeffizient von a = {rho}')

Fehler von a = [0.00449073 0.00449073]
Korrelationskoeffizient von a = -6.542726332681616e-17


## d) $A_0$ und $\delta$ berechnen (aus $a_1$ und $a_2$, mit Fehler und Korrelation)

$f(\Psi) = A_0 \cos(\Psi + \delta) = a_1 \cos(\Psi) + a_2 \sin(\Psi)$  
siehe OneNote weiter unten für folgendes:  
$A_0 = \frac{a_1}{\cos(\arctan(-a_2/a_1))}$  
$\delta = \arctan(-a_2/a_1)$   

$V\left[\begin{pmatrix}A_0 \\ \delta \end{pmatrix}\right] = J \cdot V[\hat{\vec{a}}] \cdot J^T$  
mit $J = 
\begin{pmatrix} 
    \left( \frac{a_2^2}{a_1^2} + 1 \right)^{-1/2} &
    \frac{a_2}{a_1}\left( \frac{a_2^2}{a_1^2} + 1 \right)^{-1/2} \\
    \frac{a_2}{a_1^2+a_2^2} & -\frac{a_1}{a_1^2+a_2^2}
\end{pmatrix}$

In [14]:
# A_0 und delta:
a1 = a[0]
a2 = a[1]
A_0 = a1 / np.cos(np.arctan2(-a2,a1))
delta = np.arctan2(-a2,a1) # arctan ist falsch arctan2 benötigt

print(f'{A_0 = }')
print(f'{delta = }')

A_0 = 0.08600841629560943
delta = -2.0220311136587


In [9]:
# Jacobi-Matrix:
J = [[(a2**2/a1**2+1)**-(1/2), (a2/a1)*(a2**2/a1**2+1)**-(1/2)],
     [a2/(a1**2+a2**2), -a1/(a1**2+a2**2)]]
J = np.array(J)

print('Jacobi-Matrix:')
print(f'{J = }')

Jacobi-Matrix:
J = array([[ 0.43607706, -0.89990933],
       [10.46303798,  5.07016733]])


In [10]:
# Kovarianzmatrix:
V = J @ V_a @ J.T

print('Kovarianzmatrix:')
print(f'{V = }')

Kovarianzmatrix:
V = array([[2.01666667e-05, 2.71050543e-20],
       [1.35525272e-20, 2.72616550e-03]])


In [11]:
# Fehler von a:
err = np.sqrt(np.diag(V)) 
# Korrelationskoeffizient:
rho_ = V[1,0] / err[0]**2 

print(f'Fehler von A_0 = {err[0]}')
print(f'Fehler von delta = {err[1]}')
print(f'Korrelationskoeffizient = {rho_}')

Fehler von A_0 = 0.004490731195102493
Fehler von delta = 0.05221269485613977
Korrelationskoeffizient = 6.720261399703542e-16


## Endergebnis:  

für $f(\Psi) = A_0 \cos(\Psi + \delta) = a_1 \cos(\Psi) + a_2 \sin(\Psi)$  
ergibt sich aus den gegebenen Daten mit der Methode der kleinsten Quadrate:  
$a_1 = (-0.0375 \pm 0.0045)$,  
$a_2 = (0.0774 \pm 0.0045)$,  
$A_0 = (-0.0860 \pm 0.0045)$,  
$\delta = (1.120 \pm 0.052)$

Die Korrelationskoeffizienten sind mit $\approx 10^{-20}$ nahezu 0 und würden wir nicht weiter für eine Analyse beachten.  
Zudem handelt es sich hierbei vermutlich eher um Rundungsfehler, da für $A_0$ und $\delta$ die Kovarianzmatrix nicht symmetrisch ist.